# Python functions to build a graph


## load card work for onload with Neo4j
Use the Amazon data 0601

Product co-purchasing networks data summary

    Type 	Nodes 	Edges 	Description 
    Dir 	403,394 3,387,388 Amazon product co-purchasing network from June 1 2003

    meta 	548,552 1,788,725 	meta data


Amazon product metadata: product info and all reviews on around 548,552 products.

### Once this Notebook is run there will exist

CSV files that will be used to load the database

In [1]:
#this is a text file which contains all the co-purchased products from Amazon on 06_01
file = 'C:/Users/moore/Dropbox/graphs/Amazon0601.txt'
location_gml = 'C:/Users/moore/Dropbox/graphs/Amazon.gml'
location_pickle = 'C:/Users/moore/Dropbox/graphs/Amazon.gpickle'
file_meta = 'C:/Users/moore/Dropbox/graphs/Amazon-meta.txt'
pickle_meta = 'C:/Users/moore/Dropbox/graphs/AmazonMeta.p'
location_pairs = 'C:/Users/moore/Dropbox/graphs/Amazon.p'

review_csv = 'C:/Users/moore/Dropbox/graphs/reviews.csv'
sims_csv = 'C:/Users/moore/Dropbox/graphs/sims.csv'
prods_csv = 'C:/Users/moore/Dropbox/graphs/prods.csv'
bought_csv = 'C:/Users/moore/Dropbox/graphs/bought.csv'
cats_csv = 'C:/Users/moore/Dropbox/graphs/cats.csv'
import matplotlib.pyplot as plt
import networkx as nx
import pickle

In [2]:
paths = pickle.load(open(location_pairs, 'rb')) 
meta = pickle.load(open(pickle_meta, 'rb'))  


In [68]:
with open(review_csv, 'w') as f:
    cols = 'ASIN,' + str(list(meta['1']['reviews']['reviews'][0].keys())).replace('[','').replace(']','') + '\n'
    f.write(cols)
    for key, value in meta.items():
    
        try:
            for review in meta[key]['reviews']['reviews']:
                f.write(str(meta[key]['ASIN']) +',' + str(list(review.values())).replace('[','').replace(']','') + '\n')
        except:
            continue  

In [75]:
with open(sims_csv, 'w') as f:
    cols = "prod,sim_prod\n"
    f.write(cols)
    for key, value in meta.items():    
        id = meta[key]['ASIN']
        try:
            for sim in meta[key]['similar']['sims']:
                f.write(id +',' + sim + '\n')
        except:
            continue

In [209]:
with open(prods_csv, 'w') as f:
    cols = 'ASIN, title, sales_rank, status, group, reviews, avg_rating, cats\n'
    f.write(cols)
    for key, value in meta.items():    
        id = meta[key]['ASIN']
        salesrank = ''
        title = ''
        status = 'in-active'
        group = ''
        cats = ''
        avg_rating = ''
        reviews = ''
        if ( len(meta[key].keys())) > 2:
            status = 'active'
        avail_keys = list(meta[key].keys())
        if 'title' in avail_keys:
            title = meta[key]['title'].replace(',','').replace('"','').strip()
        if 'salesrank' in avail_keys:
            salesrank = meta[key]['salesrank'].replace(',','-')
        if 'group' in avail_keys:
            group = meta[key]['group'].replace(',','-')
   
        if 'reviews' in avail_keys:
            avail_rev = list(meta[key]['reviews']['head'].keys()) 
                       
            if ('total' in avail_rev and 'avg rating' in avail_rev):
                reviews = meta[key]['reviews']['head']['total']
                avg_rating = meta[key]['reviews']['head']['avg rating']
                
        if 'categories' in avail_keys:
            cats = meta[key]['categories']['count']
            
        f.write(id +','+ title +','+ salesrank +','+ status +','+ group +','+ reviews +','+ avg_rating +','+ str(cats) + '\n')


In [155]:
with open(bought_csv, 'w') as f:
    cols = 'asin, co_pur\n'
    f.write(cols)
    for path in paths:
        x, y = path
        pair = meta[x]['ASIN'], meta[y]['ASIN']
        f.write(pair[0] +','+ pair[1] + '\n')

In [7]:
with open(cats_csv, 'w') as f:
    cols = 'ASIN,micro,sub,cat,group\n'
    f.write(cols)
    for key, value in meta.items(): 
        clean = []
        if 'categories' in meta[key].keys():
            for m in meta[key]['categories']['values']:
                ca = list(m)[:4]
                for cat in ca[1:]:
                    cleaned = cat.split('[')[0].upper().replace(',','')
                    clean.append(cleaned)
                
                if (len(clean) == 3):
                    micro = clean[2]
                    sub = clean[1]
                    cat = clean[0]
                elif (len(clean) == 2):
                    micro = clean[1]
                    sub = clean[1]
                    cat = clean[0]
                else:
                    micro = clean[0]
                    sub = clean[0]
                    cat = clean[0]
                
                f.write(meta[key]['ASIN'] + ',' + micro +',' + sub + ',' + cat +','+ meta[key]['group'].upper() + '\n')